In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import math
import folium
from folium import GeoJson, LinearColormap
from folium.features import DivIcon
from folium.plugins import HeatMap
import warnings
import branca

warnings.filterwarnings('ignore')

In [2]:
# try and reduce the amount of times you run this itll take forever.
##### only need this cell if you are wanting to check against other map features see bash script to decide what features you want
gdf = gpd.read_file('.\output.geojson')
boundaries = gpd.read_file('.\\boundaries.geojson')

# manually fix names set() ^ set()
boundaries.loc[boundaries.name == 'Juan de Fuca Electoral Area', 'name'] = 'Juan de Fuca'
boundaries.loc[boundaries.name == 'Salt Spring Island Electoral Area', 'name'] = 'Salt Spring Island'
boundaries.loc[boundaries.name == 'Southern Gulf Islands Electoral Area', 'name'] = 'Southern Gulf Islands'

def remove_times():
    # use for error in folio TypeError: Object of type Timestamp is not JSON serializable
    drop_ls = []
    for c in gdf.columns:
        if ("time" in c or "date" in c or "hour" in c):
            print(c)
            drop_ls.append(c)

    return gdf.drop(drop_ls, axis=1)

gdf = remove_times() # only use if you have an error with folium Geo.Json


check_date
survey:date


In [3]:
startDt = pd.read_csv(".\May2023.csv")
endDt = pd.read_csv(".\May2024.csv")

In [4]:
# get the center of all unique munis

def get_weight(group):
    val = startDt.neighbourhood_group.value_counts()[group] - endDt.neighbourhood_group.value_counts()[group]
    return val / max(startDt.neighbourhood_group.value_counts()[group], endDt.neighbourhood_group.value_counts()[group])

def get_center_loc(group, id=None, res_dic=None):
    # group is a data frame
    # source http://www.geomidpoint.com/example.html
    # gets the center lat lng
    # assuming the earth is perfectly round
    rads_lat = np.radians(group.latitude)
    rads_lon = np.radians(group.longitude)
    
    # Convert lat/lon (radians) to Cartesian coordinates for each location.
    X = np.cos(rads_lat) * np.cos(rads_lon)
    Y = np.cos(rads_lat) * np.sin(rads_lon)
    Z = np.sin(rads_lat)

    #find average x, y, z coords
    x = X.mean()
    y = Y.mean()
    z = Z.mean()

    # Convert average x, y, z coordinate to latitude and longitude.
    lng = math.atan2(y, x)
    hyp = np.sqrt(x*x + y*y)
    lat = math.atan2(z, hyp)
    if res_dic != None:
        res_dic[id] = [np.degrees(lat), np.degrees(lng), len(group.id.tolist())-endDt.neighbourhood_group.value_counts()[id], get_weight(id)]
    else:
        return (np.degrees(lat), np.degrees(lng),)



                          
result_start = {}
startDt.groupby(['neighbourhood_group']).apply(lambda group: get_center_loc(group=group, id=group.name, res_dic=result_start))


startDt['location'] = startDt.apply(lambda row: [row.latitude, row.longitude, get_weight(row.neighbourhood_group)], axis=1)
endDt['location'] = endDt.apply(lambda row: [row['latitude'], row['longitude'], get_weight(row.neighbourhood_group)], axis=1)


In [5]:
# break up all the startDt data into single owners and multi owners
start_count = startDt.host_id.value_counts()
multi_start = startDt[startDt.host_id.isin(start_count.index[start_count.gt(1)])]
single_start = startDt[startDt.host_id.isin(start_count.index[start_count.lt(2)])]

# break up all endDt data into multi owners and single owners
end_count = endDt.host_id.value_counts()
multi_end = endDt[endDt.host_id.isin(end_count.index[end_count.gt(1)])]
single_end = endDt[endDt.host_id.isin(end_count.index[end_count.lt(2)])]

In [6]:
# get mean price grouped by host_id
# values tracking multi_start, multi_end, single_start, single_end, singleDt, endDt
multi_start['mean_price'] = multi_start.groupby(['host_id']).price.transform('mean')
multi_end['mean_price'] = multi_end.groupby(['host_id']).price.transform('mean')

In [7]:
    
# get real center for group listings
# this function could be cleaned up
def get_group_center(group, center):
    i = 0
    for index, row in group.iterrows():
        if i == 0:
            closest = (row.latitude, row.longitude)
            i += 1
        if abs(row.latitude - center[row.host_name][0] + row.longitude - center[row.host_name][1]) < abs(closest[0]-center[row.host_name][0] + closest[1]-center[row.host_name][1]):
            closest = [row.latitude, row.longitude]
    return closest

# find closest to center listing
def find_center_listing(df):
    center_listing = {}
    center = df.groupby(['host_name']).apply(get_center_loc)
    for host in df.host_name.unique():
        center_listing[host] = get_group_center(df[df.host_name == host], center)
    
    return center_listing

multi_start_centers = find_center_listing(multi_start)
multi_end_centers = find_center_listing(multi_end)




In [80]:
map = folium.Map(location=get_center_loc(startDt), prefer_canvas=True, tiles="cartodb positron", zoom_start=9)

# create layers
start_group = folium.FeatureGroup(name="May 2022 Home •---shared-owner---• Home", control=True, show=False).add_to(map)
end_group = folium.FeatureGroup(name="May 2023 Home •---shared-owner---• Home", control=True, show=False).add_to(map)
heat_group =folium.FeatureGroup(name="% of airbnb change May 2022 to May 2023", control=True, show=False).add_to(map)

folium.LayerControl(position='bottomleft', collaped=False).add_to(map)

In [81]:
# gradient map section
# normalize weights this section is a mess and needs to be optimized
muni_weight = {}
for k, v in result_start.items():
    muni_weight[k] = v[3]

#### correct names here
muni_df = gpd.GeoDataFrame(list(muni_weight.items()), columns=['name','weight'])
geo_data = boundaries.merge(muni_df, on='name', how='left')


def weight_to_color(weight, vmin, vmax):
    cmap = plt.get_cmap('coolwarm')
    norm = mcolors.Normalize(vmin=vmin, vmax=vmax)
    rgba = cmap(norm(weight))
    return mcolors.to_hex(rgba)

# Determine the min and max weights
min_weight = geo_data['weight'].min()
max_weight = geo_data['weight'].max()


# Add GeoJson layer with popups and color mapping
folium.GeoJson(
    geo_data,
    style_function=lambda feature: {
        'fillColor': weight_to_color(feature['properties']['weight'], min_weight, max_weight),
        'color': 'black',
        'weight': 0.5,
        'dashArray': '5, 5'
    }
).add_to(heat_group)

In [82]:
# making the lines for the connections between mulit owners
def apply_lines(x, color, group, counts, start, opacity):
    # apply lines is used for both end and start groups however we keep the same centers
    if (x.host_name in multi_end_centers):
        folium.PolyLine(
            locations=[multi_end_centers[x.host_name],[x.latitude, x.longitude]],
            popup=f'Owner {x.host_name} Mean value per night ${x.mean_price:.2f} total properties {counts[x.host_name]}',
            color=color,
            weight=1,
            opacity=opacity
        ).add_to(group) 
    else:
        folium.PolyLine(
            locations=[multi_start_centers[x.host_name],[x.latitude, x.longitude]],
            color=color,
            popup=f'Owner {x.host_name} Mean value per night ${x.mean_price:.2f} total properties {counts[x.host_name]}',
            weight=1,
            opacity=opacity
        ).add_to(group),

In [83]:
multi_start.apply(lambda x: apply_lines(x, "red", start_group, multi_start.host_name.value_counts(), True, 0.5), axis=1)

0       None
2       None
4       None
7       None
9       None
        ... 
4535    None
4536    None
4537    None
4538    None
4540    None
Length: 2207, dtype: object

In [84]:
multi_end.apply(lambda x: apply_lines(x, "blue", end_group, multi_end.host_name.value_counts(), False, 0.5), axis=1)

1       None
2       None
5       None
7       None
8       None
        ... 
3973    None
3974    None
3975    None
3978    None
3979    None
Length: 1751, dtype: object

In [85]:
startDt.apply(lambda x: folium.CircleMarker(
                                location=[x.latitude, x.longitude],
                                radius=1,
                                popup=f'Price per night ${x.price}.\nHost {x.host_name}',
                                color="red",
                            ).add_to(start_group), axis=1)

0       <folium.vector_layers.CircleMarker object at 0...
1       <folium.vector_layers.CircleMarker object at 0...
2       <folium.vector_layers.CircleMarker object at 0...
3       <folium.vector_layers.CircleMarker object at 0...
4       <folium.vector_layers.CircleMarker object at 0...
                              ...                        
4536    <folium.vector_layers.CircleMarker object at 0...
4537    <folium.vector_layers.CircleMarker object at 0...
4538    <folium.vector_layers.CircleMarker object at 0...
4539    <folium.vector_layers.CircleMarker object at 0...
4540    <folium.vector_layers.CircleMarker object at 0...
Length: 4541, dtype: object

In [86]:
endDt.apply(lambda x: folium.CircleMarker(
                                location=[x.latitude, x.longitude],
                                radius=.5,
                                popup=f'Price per night ${x.price}.\nHost {x.host_name}',
                                color="blue",
                            ).add_to(end_group), axis=1)

0       <folium.vector_layers.CircleMarker object at 0...
1       <folium.vector_layers.CircleMarker object at 0...
2       <folium.vector_layers.CircleMarker object at 0...
3       <folium.vector_layers.CircleMarker object at 0...
4       <folium.vector_layers.CircleMarker object at 0...
                              ...                        
3977    <folium.vector_layers.CircleMarker object at 0...
3978    <folium.vector_layers.CircleMarker object at 0...
3979    <folium.vector_layers.CircleMarker object at 0...
3980    <folium.vector_layers.CircleMarker object at 0...
3981    <folium.vector_layers.CircleMarker object at 0...
Length: 3982, dtype: object

In [87]:
# sbs = folium.plugins.SideBySideLayers(layer_left=start_group, layer_right=end_group)
# sbs.add_to(map)
colormap = branca.colormap.LinearColormap(colors=['blue', 'red'],vmin=min_weight, vmax=max_weight,)
colormap.caption = '% of airbnb change'
colormap.add_to(map)

map.save('map.html')